# Numerical Sorting for Decision Trees
*© Copyright James Dellinger, 2022-2025. All rights reserved.*

Numerical split finding is where decision tree growing algorithms spend the bulk of their time, and so optimizing this aspect of the codebase will yield the biggest bang for the buck if one wishes to speed up their decision tree implementation.

Specifically, sorting numerical feature values in ascending order is the first step toward finding a numerical split, and so in this cohort of experiments I've tested out various pure Python implementations of the [Introsort](https://en.wikipedia.org/wiki/Introsort) sorting algorithm that's used in both [Sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) and [Ranger](https://github.com/imbs-hl/ranger/blob/5f71872d7b552fd2cf652daab92416f52976df86/src/Data.cpp#L227) (when using "SmallQ" splitting instead of pre-sorting numerical features).

#### Learning by doing
As someone who is in the midst of creating his own decision trees library from scratch, my aim in all this was to get intimately acquainted with the introsort algorithm and all its components, and to understand why each of these sub-routines is indispensable. In addition to reading the pertinent academic papers, I found that walking through other popular libraries' (Numpy, libstdc++, etc.) introsort implementations and then reproducing them on my own in Python did the most to deepen my intuition and solidify my understanding.

Surveying then faithfully recreating in Python the different approaches that "the pros" have taken in implementing introsort to run performantly gave me the exposure and context to then be able to pick and choose, "a la carte," the algorithms, styles, conventions, and tricks that most closely matched my way of thinking.

#### The final product
I ultimately distilled my code down to a single pure Python implementation of introsort, which I then translated to Cython. When I went to benchmark the speed of my implementation against Sklearn's Cython version of introsort, I was surprised to find that over the past few years, Sklearn has used no fewer than [two](https://github.com/scikit-learn/scikit-learn/blob/95d4f0841d57e8b5f6b2a570312e9d832e69debc/sklearn/tree/_splitter.pyx#L515) [different](https://github.com/scikit-learn/scikit-learn/blob/ddc8210c7716cb287f167e502fb716e894cec6fc/sklearn/utils/_sorting.pyx#L19) sorting algorithms in its decision trees module, and that neither of them have been *completely consistent* with what [David Musser](https://en.wikipedia.org/wiki/David_Musser) specified in his [1997 paper](https://www.cs.rpi.edu/~musser/gp/introsort.ps) that introduced the Introsort algorithm. 

Sklearn's most recent sorting logic, which was initially touted as a new and improved refactor, contained an achilles heel that would shortly force the team to have to revert to the older implementation. Later in this notebook I walk through what happened there, and why their new sorting implementation broke, and how the moral of that story is that Musser's introsort is as necessary as ever.

Thankfully, other major implementations haven't made similar missteps. Ranger for example, being written in C++, uses `std::sort`, which is faithful to what Musser recommended. Numpy's `np.sort` is also nearly identical to Musser's introsort, with the exception of one [minor difference](https://github.com/numpy/numpy/blob/5ffb84c3057a187b01acdeaa628137193df12098/numpy/core/src/npysort/quicksort.cpp#L197), on which I'll further elaborate in the section just below.

## Introsort
Before I discuss in greater detail what's been going on over at Sklearn, here's an summary of Musser's conception of and justification for introsort, per his 1997 paper:

1. Use an efficient implementation of Median-of-Three Quicksort ([Bentley & McIlroy, 1993](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.14.8162&rep=rep1&type=pdf)) to begin recursively sorting a list of numbers. "Efficient" here means:
     1. Swap equal elements. 
     2. A decent quicksort implementation will recurse on only the smaller branch of a partition and allow the larger sub-partition to be handled in the next go-round of the current while-loop. This ensures that stack depth will be no larger than $O(\log N)$. Otherwise, it would be $O(N)$. However, this is not necessary for introsort because introsort already enforces a simple max recursion limit that on its own prevents recursion from exceeding a stack depth of $O(\log N)$.
     
        For example, Numpy manages to break this requirement for quicksort implementations by [placing the larger side of a partition on the stack](https://github.com/numpy/numpy/blob/5ffb84c3057a187b01acdeaa628137193df12098/numpy/core/src/npysort/quicksort.cpp#L197) and then iterating over the samples that remain inside the smaller partition. But crucially, this won't hurt Numpy's performance since the fact that it employs a full-fledged introsort implementations means that either way, stack depth will not exceed $O(\log N)$.
     3. Set a minimum length threshold for sorting subproblems, where sequences shorter than this threshold are not recursed (or iterated) over using quicksort, but are left alone until all longer sequences have been addressed. 
     4. These short sequences are then mopped up in one final pass over all numbers using [insertion sort](https://www.geeksforgeeks.org/insertion-sort/). Stanford Computer Science Department Senior Lecturer [Keith Schwarz](https://keithschwarz.com/) [explains](https://www.keithschwarz.com/interesting/code/introsort/Introsort.hh.html) that this is because insertion sort runs faster than quicksort on average ($O(N)$ vs. $O(N\log N)$, where $N$ is the number of items to sort) when all elements are no further than a given maximum distance from what will be their final sorted locations. In the libstdc++ implementation of `std::sort` this threshold is [set to](https://github.com/gcc-mirror/gcc/blob/d9375e490072d1aae73a93949aa158fcd2a27018/libstdc%2B%2B-v3/include/bits/stl_algo.h#L1864) $16$. In Numpy's implementation, $15$ [is used](https://github.com/numpy/numpy/blob/5ffb84c3057a187b01acdeaa628137193df12098/numpy/core/src/npysort/quicksort.cpp). 
2. Keep track of recursion depth and cease any further calls to quicksort once a maximum depth of $2*\log_2 N$ has been exceeded. Switch over to heapsort to finish all remaining samples in the branch. $2*\log_2 N$ was chosen as the depth threshold based on Musser's empirical results.
3. The motivation behind heapsort's inclusion: while quicksort has an average computing time of $O(N\log N)$, in its worst-case scenario it goes quadratic with a time of $O(N^2)$. Although heapsort is on average two to five times slower than quicksort, its worst-case computing time is only $O(N\log N)$. Switching from quicksort over to heapsort once recursion gets too deep gives us all the speed benefits of median-of-three quicksort, while protecting us in those situations when quicksort would lag well behind heapsort.

#### Aside: Where to do insertion sort
Similar to optimized implementations of median-of-three quicksort, Musser's introsort runs one final pass of insertion sort over the entire list of numbers after all recursive calls have been completed. However, some practitioners might wonder whether it would be better to initiate several calls to insertion sort throughout the sorting process, making one call on each branch where partition size had decreased below the minimum length threshold. 

Musser also investigated this in his paper. He knew that running one final pass of insertion sort could double the number of memory [cache misses](https://hazelcast.com/glossary/cache-miss/), but that older computers didn't have that much memory to begin with and it was virtually certain that the cumulative cache miss penalties incurred by running insertion sort only once wouldn't be greater than the overhead of running insertion sort at the end of each and every recursion branch.

However, he hypothesized that for of modern machines with more memory to sort longer sequences, the cumulative cache miss penalty of running insertion once might actually be more significant than the overhead of running it several times inside introsort's recursive loop. In other words, Musser believed there was a chance that moving insertion sort inside his sorting function's recursive loop would speed up sorting performance on computers with large memory capacities.

Nonetheless, Musser's experiments demonstrated that even on modern machines, calling insertion sort just once near the end of sorting was *almost always faster* than calling it many times throughout sorting.

#### But how much do we really need heapsort?
When Bentley and McIlroy published their median-of-three partitioning strategy in 1993, they fixed one of the greatest flaws of the classic quicksort algorithm -- namely, that its compute time would go quadratic when attempting to sort a sequence of numbers that was nearly already perfectly sorted. 

However, in his 1997 paper Musser identified a general category of numerical sequences that he called "Median-of-3 Killer Sequences." Ideally, median-of-three partitioning should split a sequence of numbers into two sub-sequences of roughly identical length. Median-of-three killer sequences, on the other hand, will force median-of-three quicksort to split a sequence of $N$ numbers into sub-sequences of $2$ and $N-2$ numbers. This triggers an excess of partitions created during sorting, which in turn leads to deeper recursion and ultimately causes the sorting algorithm to go quadratic and require $O(N^2)$ time to complete.

Musser acknowledged that in everyday practice, these killer sequences were not nearly as commonplace as the "nearly sorted" lists of numbers that cause classic quicksort implementations to go quadratic. However, he still believed that it was unreasonable to presume median-of-three killer sequences would be equally as rare as any other type of sequence found under a uniform distribution of numerical sequence permutations. Indeed, Musser was persuaded by [the 1992 work](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.29.2135&rep=rep1&type=pdf) of Ming Li and Paul Vitanyi and thought that the simple fact that median-of-three killer sequences can be produced by a short computer program warranted the assumption that these sequences occur under a universal distribution, as opposed to a uniform distribution, and would thus be more common than otherwise expected. 

#### An Example of a "Median-of-3 Killer" Sequence
Here's the recipe for generating a median-of-three killer:
1. Take a consecutive sequence of $1, ..., 2k$ integers where $2k$ is divisible by $4$.
2. Permute the sequence such that the new order is:

    $1,\;k+1,\;3,\;k+3,\;5,\;...,\;2k-3,\;k-1,\;2k-1,\;2,\;4,\;6,\;...,\;2k-2,\;2k$
    
Let's generate one now:

In [1]:
def generate_med3_killer(N):
    """Create a median-of-3 killer sequence (Musser, 1997).
    
    Arguments:
        N (int): The desired length of the sequence. Must be divisble by 4.
              
    Returns: 
        The median-of-3 killer sequence.
    """
    if N%4 != 0: 
        print('A median-of-3 killer sequence\'s length must be divisible by 4.')
    else:
        k = int(N/2)     
        return [i if i%2 == 1 else k+i-1 for i in range(1, k+1)] + [i for i in range(1, 2*k + 1) if i%2 == 0]

In [2]:
generate_med3_killer(20)

[1, 11, 3, 13, 5, 15, 7, 17, 9, 19, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

## Implementing Quicksort in Python and then breaking it
To observe how certain killer sequences can make median-of-three quicksort go quadratic, let's implement a pure Python version of quicksort that employs all the standard optimizations, save for the inclusion of heapsort. 

#### Helper function

In [3]:
def swap(items, i, j):
    items[i], items[j] = items[j], items[i]

#### Finding the median-of-three pivot point
There are two ways this is commonly done.

In their [1993 paper](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.14.8162&rep=rep1&type=pdf) that introduced median-of-three quicksort, Bentley and McIlroy proposed the following algorithm for finding the median-of-three pivot point. The justification behind this algorithm was that it performed an average of $8/3$ comparisons to the median of three values, and that it didn't require any swaps.

In [4]:
def med_three_bentley_mcilroy(items, n):
    """Get the median-of-three pivot point of a list of elements. 
    From Bentley and McIlroy, 1993.
    Borrows from sklearn implementation at: 
        https://github.com/scikit-learn/scikit-learn/blob/95d4f0841d57e8b5f6b2a570312e9d832e69debc/sklearn/tree/_splitter.pyx#L493
    
    Arguments:
        items (list): The numbers to be sorted.
        n      (int): The number of items to be sorted. 
    """
    a, b, c = items[0], items[n//2], items[n-1]
    if a < b:
        if   b < c: return b
        elif a < c: return c
        else:       return a
    elif b < c:
        if   a < c: return a
        else:       return c
    else:           return b

Sklearn [used to](https://github.com/scikit-learn/scikit-learn/blob/95d4f0841d57e8b5f6b2a570312e9d832e69debc/sklearn/tree/_splitter.pyx#L493) use such an algorithm, however they've [recently switched](https://github.com/scikit-learn/scikit-learn/blob/ddc8210c7716cb287f167e502fb716e894cec6fc/sklearn/utils/_sorting.pyx#L62) to an implementation that has a lower maximum worst-case number of comparisons but that requires swaps. It turns out that several major libraries, such as [Numpy](https://github.com/numpy/numpy/blob/5ffb84c3057a187b01acdeaa628137193df12098/numpy/core/src/npysort/quicksort.cpp#L169), also use swaps to find the median value.

But I found that the structure and variable naming of the [libstdc++](https://github.com/gcc-mirror/gcc/blob/d9375e490072d1aae73a93949aa158fcd2a27018/libstdc%2B%2B-v3/include/bits/stl_algo.h#L78) sorting algorithm was the most interesting approach and intuitively made the most sense to me, so I've modeled my Python implementation that follows below on what they did.

In [5]:
def med_three(items, first, last):
    """Find the median-of-three pivot point of the second through final 
    items of a list of numbers. Once identified, the pivot is moved to 
    the front of the list. Borrows from libstdc++ implementation at: 
        https://github.com/gcc-mirror/gcc/blob/d9375e490072d1aae73a93949aa158fcd2a27018/libstdc%2B%2B-v3/include/bits/stl_algo.h#L78
    
    Arguments:
        items      (list): The numbers to be sorted.
        first, last (int): The range of items to be sorted. 
    """
    middle = first + (last - first)//2
    second = first + 1
    last -= 1
    if items[second] < items[middle]:
        if items[middle] < items[last]:
            swap(items, first, middle)    
        elif items[second] < items[last]:
            swap(items, first, last)         
        else:                        
            swap(items, first, second)
    elif items[second] < items[last]:
        swap(items, first, second)
    elif items[middle] < items[last]:
        swap(items, first, last)
    else:
        swap(items, first, middle)

In the above implementation, the median value of the second through final numbers (`items[2:]`) will be placed at the beginning of the `items` list. Note that this isn't the only way to do this; an alternative approach could be to place the median value of the entire sequence at the end of the list of numbers, which is what Sklearn and Numpy both do.

The advantage to libstdc++'s approach is that it makes a maximum of one swap while Numpy and Sklearn might make two or even three swaps.

This next function takes the same sequence of numbers that now holds the median value at its beginning, and then executes some swaps so that elements that are smaller than the median pivot value will precede all the elements that are greater than it. The median pivot will remain at the beginning of the list. 

When this reshuffling is all finished, the function returns the index of the first item that is greater than or equal to the median pivot value. When quicksort is ready to split the current list of numbers into two smaller partitions, this index will indicate where the right-hand partition begins.

In [6]:
def partition(items, first, last, pivot):
    """Group numbers less than the pivot value together on the left and
    those that are greater on the right. Find the index that separates
    these two groups, which will belong to the first item that is greater
    than or equal to the pivot. Borrows from libstdc++ implementation at: 
        https://github.com/gcc-mirror/gcc/blob/d9375e490072d1aae73a93949aa158fcd2a27018/libstdc%2B%2B-v3/include/bits/stl_algo.h#L1885
    
    Arguments:
        items      (list): The numbers to be sorted.
        first, last (int): The range of items to be sorted. 
        pivot       (int): Index holding the median pivot value.
        
    Returns:
        Index of cut point used to partition the items into two smaller sequences.
    """
    while True:
        while first < len(items) and items[first] < items[pivot]:
            first += 1                      # Get index of first item greater than or equal to median-of-three pivot. 
        last -= 1
        while items[pivot] < items[last]:
            last -= 1                       # Get index of last item less than or equal to the pivot.
        if not (first < last): 
            return first                    # After swaps are done, return index of first item in right partition.
        
        swap(items, first, last)            # Swap the first item greater than or equal to the pivot with the
                                            # last item less than or equal to the pivot. 
        first += 1 

`quicksort_loop()` contains all of quicksort's recursive logic. For each larger sub-partition, it iteratively finds the median of three and then further divides that group of numbers into two smaller partitions. As Musser recommended, recursive calls are made only on smaller sub-partitions in order to keep stack depth within $O(\log N)$.

In [7]:
def quicksort_loop(items, first, last):
    """The recursive heart of the median-of-three quicksort algorithm.
    
    Arguments:
        items      (list): The numbers to be sorted.
        first, last (int): The range of items to be sorted. 
    """
    MIN_SIZE_THRESH = 16
    while last-first > MIN_SIZE_THRESH:
        med_three(items, first, last)
        cut = partition(items, first+1, last, first)
        if last - cut < cut - first:
            quicksort_loop(items, cut, last)
            last = cut
        else:
            quicksort_loop(items, first, cut)
            first = cut

I liked the way Numpy wrote [their version](https://github.com/numpy/numpy/blob/5ffb84c3057a187b01acdeaa628137193df12098/numpy/core/src/npysort/quicksort.cpp#L211) of insertion sort, and so I've reimplemented it here.

In [8]:
def insertion_sort(items, first, last):
    """Follows the spirit of the Numpy implementation at: 
        https://github.com/numpy/numpy/blob/5ffb84c3057a187b01acdeaa628137193df12098/numpy/core/src/npysort/quicksort.cpp#L211
    
    Arguments:
        items      (list): The numbers to be sorted.
        first, last (int): The range of items to be sorted. 
    """
    for i in range(first+1, last):
        j = i
        val = items[i]
        k = i - 1
        while (j > first) and (val < items[k]):
            items[j] = items[k]
            j-=1
            k-=1
        items[j] = val

For all partitions larger than `MIN_SIZE_THRESH = 16`, quicksort will run the median-of-three sorting algorithm. Insertion sort is run in one final pass across the entire list of numbers to be sorted.

In [9]:
def quicksort(items, first, last):
    """Median-of-three quicksort (Bentley & McIlroy, 1993). Contains all the usual 
    optimizations:
        - Swap equal elements.
        - Only process partitions longer than the minimum size threshold.
        - When a new partition is made, recurse on the smaller half and 
          iterate over the larger half.
        - Make a final pass with insertion sort over the entire list.

    Arguments:
        items      (list): The numbers to be sorted.
        first, last (int): The range of items to be sorted. 
    """
    quicksort_loop(items, first, last)
    insertion_sort(items, first, last)

#### What Happens When Quicksort Goes Quadratic

In [10]:
import numpy as np
rng = np.random.default_rng(42)

In [11]:
# Random sequence with replacement under uniform distribution.
%timeit quicksort(rng.choice(np.arange(1,225001), 225000), 0, 225000)

1.79 s ± 87.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
# Med 3 killer.
%timeit quicksort(generate_med3_killer(225000), 0, 225000)

4min 52s ± 512 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Incredibly, quicksort takes over *150 times longer* to sort median-of-three killer sequence containing 225,000 integers compared to sorting 225,000 integers randomly selected from a uniform distribution!

## The Importance of Heapsort
We just observed what happens when quicksort blasts past its average computing time of $O(N\log N)$ and goes parabolic and hits its worst-case scenario where it requires $O(N^2)$ time to sort numbers. A greater than hundred-fold slowdown is no laughing matter, and the possibility of quicksort grinding to a crawl like this is precisely what spurred Musser to invent introsort.

Introsort was so named because Musser designed the algorithm to be *introspective* about how efficiently quicksort's median-of-three partitioning scheme was working. Ideally, a median-of-three partition should split a sequence of numbers into two roughly similar-sized sub-partitions. And when this is the case, quicksort's recursion will not grow terribly deep relative to the number of items to be sorted. However, should the median-of-three rule begin making several grossly unbalanced splits by sending just a few items to one sub-partition while funnelling all the rest to the other (the kind of behavior triggered by a median-of-three killer sequence), then recursion depth will snowball and cause compute time to go quadratic.

What introsort does is keep track of recursion depth, and when recursion does get too deep (because too many inefficient partitions have been made), introsort stops quicksort in its tracks and forces it to pass the baton off to the heapsort algorithm to finish things up. At this point, instead of the current sub-partition getting further sub-divided by median-of-three, heapsort will sort all the numbers inside this sub-partition in one fell swoop. 

Although heapsort tends to be a bit slower than quicksort at its best, heapsort's worst-case compute time is only $O(N\log N)$. Therefore, switching over to heapsort whenever recursion gets too deep is a surefire way to prevent quicksort from going quadratic on us. 

#### Killing Sklearn's `simultaneous_sort()`
Unfortunately, as of late March, 2022, Sklearn had begun using a [refactored](https://github.com/scikit-learn/scikit-learn/pull/22868) sorting algorithm that no longer monitors recursion depth and no longer falls back on heapsort when recursion grows too deep.

I'll shortly demonstrate how to implement heapsort in Python and use it in introsort as Musser intended. But first, in order to further underscore my opinion that it may not have been such a good idea for Sklearn to remove heapsort from the numerical sorting algorithm that they use to grow decision trees, I'll run the median-of-three killer experiment once more, but this time using none other than Sklearn's newly-refactored sorting algorithm.

To pull this off, just below I've copied and pasted in below the Cython code that can be found in Sklearn's [_sorting.pyx](https://github.com/scikit-learn/scikit-learn/blob/ddc8210c7716cb287f167e502fb716e894cec6fc/sklearn/utils/_sorting.pyx#L19) file. Note that each line of code has been left precisely identical to what's currently on Sklearn's github. My only addition was a lightweight wrapper function that grabs the pointers to the arguments' Numpy arrays, so that these can be passed to Sklearn's Cython logic. 

In [13]:
%load_ext Cython

In [14]:
%%cython
# cython: wraparound=False, boundscheck=False, cdivision=True, initializedcheck=False

# Following two imports and typedef not in Sklearn's original .pyx file,
# but necessary for the code to compile here.
import numpy as np
cimport numpy as np
ctypedef np.intp_t ITYPE_t

############## Beginning of sorting code pasted in from SKlearn's _sorting.pyx ##############
from cython cimport floating

cdef inline void dual_swap(floating* darr, ITYPE_t *iarr, ITYPE_t a, ITYPE_t b,) nogil:
    """Swap the values at index a and b of both darr and iarr"""
    cdef floating dtmp = darr[a]
    darr[a] = darr[b]
    darr[b] = dtmp

    cdef ITYPE_t itmp = iarr[a]
    iarr[a] = iarr[b]
    iarr[b] = itmp

cdef int simultaneous_sort(floating* values, ITYPE_t* indices, ITYPE_t size,) nogil:
    """
    Perform a recursive quicksort on the values array as to sort them ascendingly.
    This simultaneously performs the swaps on both the values and the indices arrays.
    The numpy equivalent is:
        def simultaneous_sort(dist, idx):
             i = np.argsort(dist)
             return dist[i], idx[i]
    Notes
    -----
    Arrays are manipulated via a pointer to there first element and their size
    as to ease the processing of dynamically allocated buffers.
    """
    # TODO: In order to support discrete distance metrics, we need to have a
    # simultaneous sort which breaks ties on indices when distances are identical.
    # The best might be using a std::stable_sort and a Comparator which might need
    # an Array of Structures (AoS) instead of the Structure of Arrays (SoA)
    # currently used.
    cdef:
        ITYPE_t pivot_idx, i, store_idx
        floating pivot_val

    # in the small-array case, do things efficiently
    if size <= 1:
        pass
    elif size == 2:
        if values[0] > values[1]:
            dual_swap(values, indices, 0, 1)
    elif size == 3:
        if values[0] > values[1]:
            dual_swap(values, indices, 0, 1)
        if values[1] > values[2]:
            dual_swap(values, indices, 1, 2)
            if values[0] > values[1]:
                dual_swap(values, indices, 0, 1)
    else:
        # Determine the pivot using the median-of-three rule.
        # The smallest of the three is moved to the beginning of the array,
        # the middle (the pivot value) is moved to the end, and the largest
        # is moved to the pivot index.
        pivot_idx = size // 2
        if values[0] > values[size - 1]:
            dual_swap(values, indices, 0, size - 1)
        if values[size - 1] > values[pivot_idx]:
            dual_swap(values, indices, size - 1, pivot_idx)
            if values[0] > values[size - 1]:
                dual_swap(values, indices, 0, size - 1)
        pivot_val = values[size - 1]

        # Partition indices about pivot.  At the end of this operation,
        # pivot_idx will contain the pivot value, everything to the left
        # will be smaller, and everything to the right will be larger.
        store_idx = 0
        for i in range(size - 1):
            if values[i] < pivot_val:
                dual_swap(values, indices, i, store_idx)
                store_idx += 1
        dual_swap(values, indices, store_idx, size - 1)
        pivot_idx = store_idx

        # Recursively sort each side of the pivot
        if pivot_idx > 1:
            simultaneous_sort(values, indices, pivot_idx)
        if pivot_idx + 2 < size:
            simultaneous_sort(values + pivot_idx + 1,
                              indices + pivot_idx + 1,
                              size - pivot_idx - 1)
    return 0

################# End of sorting code pasted in from SKlearn's _sorting.pyx #################
    
def skl_sort_new(np.ndarray[np.float64_t, ndim=1, mode="c"] values, 
                 np.ndarray[np.intp_t, ndim=1, mode="c"] indices, int n):
    """Wrapper function for my Cython implementation of `introsort()`. 
    Followed Laura Mendoza's guide on how to have Cython access
    a Numpy array using the C pointer:
        https://members.loria.fr/LMendoza/link/Cython_speedup_notes.html#Working-with-numpy-arrays-in-I/O
    
    Arguments:
        values (Numpy array, float64): The numbers to be sorted.
        indices    (Numpy array, int): Contains the index of each value in `values`.
        n                       (int): Count of numbers to be sorted.
    """
    cdef np.float64_t* values_ptr
    cdef np.intp_t* indices_ptr
    values_ptr = <np.float64_t*> values.data
    indices_ptr = <np.intp_t*> indices.data
    simultaneous_sort(values_ptr, indices_ptr, n)

#### Aside: In case you're wondering why Sklearn's sorting function expects two arrays
Due to the way Sklearn's decision tree implementation finds numerical splits, its sorting algorithm expects to receive not only a 1-d array containing the feature values to be sorted, but also second 1-d array of identical length that contains the indices of the training set rows from which these values were drawn. During the sorting process, row indices get swapped with each other as their corresponding values get swapped. In other words, the feature values are sorted in-place as their row indices get argsorted in-place. 

On page 110 of his [thesis](https://arxiv.org/pdf/1407.7502.pdf) Gilles Louppe explains that depending on the number of candidate split features and the amount of samples selected, this kind of dual swap approach might not necessarily be slower than the alternate approach pre-sorting feature values that was used by Breiman and Cutler in their Fortran Random Forests implementation. 

Now, on to the median-of-three killer experiment:

In [15]:
rng = np.random.default_rng(42)

In [16]:
%timeit skl_sort_new(rng.uniform(np.arange(1,225001), 225000), np.array([i for i in range(225000)]), 225000)

34.7 ms ± 949 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
%timeit skl_sort_new(np.array(generate_med3_killer(225000), dtype='float64'), np.array([i for i in range(225000)]), 225000)

8.31 s ± 87.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Incredibly, Sklearn's refactored Cython sorting code takes well over 200 times longer to sort a median-of-three killer sequence than it takes to sort a randomly drawn sequence of numbers!

## How Sklearn's Old Sorting Logic Fares
I'm curious to see whether or not the old sorting logic that Sklearn used before the March, 2022 refactor will also undergo a several hundred-fold slowdown when it's faced with a killer sequence so I've gone ahead and pasted in the relevant Cython code below, just as [it appears](https://github.com/scikit-learn/scikit-learn/blob/95d4f0841d57e8b5f6b2a570312e9d832e69debc/sklearn/tree/_splitter.pyx#L477) in the original source code file.

In [18]:
%%cython
# cython: wraparound=False, boundscheck=False, cdivision=True, initializedcheck=False

# Following two imports and two typedefs not in Sklearn's original .pyx file,
# but necessary for the code to compile here.
import numpy as np
cimport numpy as np
ctypedef np.npy_intp SIZE_t
ctypedef np.float64_t DTYPE_t

# Also needed following three lines from Sklearn's old _utils.pyx file at:
# https://github.com/scikit-learn/scikit-learn/blob/95d4f0841d57e8b5f6b2a570312e9d832e69debc/sklearn/tree/_utils.pyx#L77
from libc.math cimport log as ln
cdef inline double log(double x) nogil:
    return ln(x) / ln(2.0)

# Note all code pasted in below remains entirely as it appears  on Sklearn's github. 
# With the exception that in two places I had to cast the results of a division
# operation to an int because the Cython compiler was interpreting as a Double.

########### Beginning of sorting code pasted in from SKlearn's old _splitter.pyx ###########

# Sort n-element arrays pointed to by Xf and samples, simultaneously,
# by the values in Xf. Algorithm: Introsort (Musser, SP&E, 1997).
cdef inline void sort(DTYPE_t* Xf, SIZE_t* samples, SIZE_t n) nogil:
    if n == 0:
        return
    cdef int maxd = 2 * <int>log(n)
    introsort(Xf, samples, n, maxd)


cdef inline void swap(DTYPE_t* Xf, SIZE_t* samples,
        SIZE_t i, SIZE_t j) nogil:
    # Helper for sort
    Xf[i], Xf[j] = Xf[j], Xf[i]
    samples[i], samples[j] = samples[j], samples[i]


cdef inline DTYPE_t median3(DTYPE_t* Xf, SIZE_t n) nogil:
    # Median of three pivot selection, after Bentley and McIlroy (1993).
    # Engineering a sort function. SP&E. Requires 8/3 comparisons on average.
    
    # James edit: Had to cast (n/2) to <int> because Cython was converting to Double.
    cdef DTYPE_t a = Xf[0], b = Xf[<int>(n / 2)], c = Xf[n - 1]
    if a < b:
        if b < c:
            return b
        elif a < c:
            return c
        else:
            return a
    elif b < c:
        if a < c:
            return a
        else:
            return c
    else:
        return b


# Introsort with median of 3 pivot selection and 3-way partition function
# (robust to repeated elements, e.g. lots of zero features).
cdef void introsort(DTYPE_t* Xf, SIZE_t *samples,
                    SIZE_t n, int maxd) nogil:
    cdef DTYPE_t pivot
    cdef SIZE_t i, l, r

    while n > 1:
        if maxd <= 0:   # max depth limit exceeded ("gone quadratic")
            heapsort(Xf, samples, n)
            return
        maxd -= 1

        pivot = median3(Xf, n)

        # Three-way partition.
        i = l = 0
        r = n
        while i < r:
            if Xf[i] < pivot:
                swap(Xf, samples, i, l)
                i += 1
                l += 1
            elif Xf[i] > pivot:
                r -= 1
                swap(Xf, samples, i, r)
            else:
                i += 1

        introsort(Xf, samples, l, maxd)
        Xf += r
        samples += r
        n -= r


cdef inline void sift_down(DTYPE_t* Xf, SIZE_t* samples,
                           SIZE_t start, SIZE_t end) nogil:
    # Restore heap order in Xf[start:end] by moving the max element to start.
    cdef SIZE_t child, maxind, root

    root = start
    while True:
        child = root * 2 + 1

        # find max of root, left child, right child
        maxind = root
        if child < end and Xf[maxind] < Xf[child]:
            maxind = child
        if child + 1 < end and Xf[maxind] < Xf[child + 1]:
            maxind = child + 1

        if maxind == root:
            break
        else:
            swap(Xf, samples, root, maxind)
            root = maxind


cdef void heapsort(DTYPE_t* Xf, SIZE_t* samples, SIZE_t n) nogil:
    cdef SIZE_t start, end

    # heapify
    # James edit: Had to cast (n-2)/2 to <int> because Cython was converting to Double.
    start = <int>((n - 2) / 2) 
    end = n
    while True:
        sift_down(Xf, samples, start, end)
        if start == 0:
            break
        start -= 1

    # sort by shrinking the heap, putting the max element immediately after it
    end = n - 1
    while end > 0:
        swap(Xf, samples, 0, end)
        sift_down(Xf, samples, 0, end)
        end = end - 1

############## End of sorting code pasted in from SKlearn's old _splitter.pyx ##############
    
def skl_sort_old(np.ndarray[np.float64_t, ndim=1, mode="c"] Xf, 
                 np.ndarray[np.intp_t, ndim=1, mode="c"] samples, int n):
    """Wrapper function for my Cython implementation of `introsort()`. 
    Followed Laura Mendoza's guide on how to have Cython access
    a Numpy array using the C pointer:
        https://members.loria.fr/LMendoza/link/Cython_speedup_notes.html#Working-with-numpy-arrays-in-I/O
    
    Arguments:
        Xf     (Numpy array, float64): The numbers to be sorted.
        samples    (Numpy array, int): Contains the index of each value in `values`.
        n                       (int): Count of numbers to be sorted.
    """
    cdef np.float64_t* Xf_ptr
    cdef np.intp_t* samples_ptr
    Xf_ptr = <np.float64_t*> Xf.data
    samples_ptr = <np.intp_t*> samples.data
    sort(Xf_ptr, samples_ptr, n)

So let's see how Sklearn's old `sort()` implementation handles median-of-three killer sequences:

In [19]:
%timeit skl_sort_old(rng.uniform(np.arange(1,225001), 225000), np.array([i for i in range(225000)]), 225000)

39.1 ms ± 970 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
%timeit skl_sort_old(np.array(generate_med3_killer(225000), dtype='float64'), np.array([i for i in range(225000)]), 225000)

72.2 ms ± 1.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


This, ladies and gentlemen, is the difference between $O(N\log N)$ and $O(N^2)$ compute time -- Sklearn's deprecated sorting logic was *only twice as slow* when sorting a median-of-three killer sequence. This is nowhere near as bad as the refactored code's two-hundred fold slowdown!

#### So why did Sklearn get rid of heapsort?
Indeed, why would the Sklearn team insert an achilles' heel into their sorting routine? I can only assume they are either not aware of median-of-three killers or perhaps, unlike Musser, they believe the likelihood of a group of numbers drawn from a dataset column would causing quicksort to go quadratic is so infinitessimally small that it's not worth planning for.

After the new sorting function was merged, the reviewers touted a [15% speedup](https://twitter.com/thomasjpfan/status/1506043093107318788) thanks to the new code. However, this was based on a [benchmark test](https://github.com/scikit-learn/scikit-learn/pull/22868) that created a synthetic dataset using [the normal distribution, randomness, and other heuristics](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html). Needless to say, none of the columns made quicksort go quadratic.

#### Aside: But where did the refactored version's "15%" speed-up come from?
Notice that when sorting non-killer sequences of random numbers, that Sklearn's new sorting logic completes the task in 34.7ms, while the older implementation required 39.1 ms. In this case, the newer code was 11% faster.

If you look carefully over Sklearn's old sorting code, you'll notice that contrary to Musser's design, Sklearn had omitted insertion sort from its implementation of introsort. And while the new, refactored code still doesn't use a full-fledged implementation insertion sort, it does specify how to handle partitions with just two or three numbers. In their 1993 paper that introduced median-of-three quicksort, Bentley & McIlroy acknowledged that quicksort has severe overhead when run on small partitions, and this was why they recommended using insertion sort on such small sequences.

I believe it's reasonable that the lack of any strategy to handle small partitions may have hampered Sklearn's prior sorting implemention. And thus, it's my hypothesis is that the introduction of the logic to handle super small partitions of 3 numbers or less may well be responsible for the improved performance (on non-killer sequences) that convinced the Sklearn reviewers that the refactored sorting algorithm was the way to go.

## Implementing Introsort in Python
So now that I've driven home the point that heapsort should be a non-negotiable part of a quicksort implementation, let's transform the Python `quicksort()` function I wrote above into a fully-fledged version of Musser's `introsort()`!

#### Heapsort
Heapsort was invented by [J. W. J. Williams](https://en.wikipedia.org/wiki/J._W._J._Williams) in 1964 and has both an avaerage and worst case compute time of $O(N\log N)$. When maximum recursion depth is reached, introsort stops using median-of-three and instead finishes sorting the current partition using heapsort.

Here are a couple different ways of implementing heapsort, each inspired by a different popular library.

#### Sklearn's [approach](https://github.com/scikit-learn/scikit-learn/blob/95d4f0841d57e8b5f6b2a570312e9d832e69debc/sklearn/tree/_splitter.pyx#L571)

In [21]:
def sift_down(items, first, last, parent):
    """Swap a heap's item with one of its children if that child's value is 
    greater than or equal to the item's value. From Williams, 1964.
    Modeled after Sklearn's implementation at:
        https://github.com/scikit-learn/scikit-learn/blob/95d4f0841d57e8b5f6b2a570312e9d832e69debc/sklearn/tree/_splitter.pyx#L548
    
    Arguments:
        items      (list): The numbers to be sorted.
        first, last (int): The range of items to be sorted.
        parent      (int): Index of the parent item.
    """
    while True:
        child = 2*parent + 1 - first
        maximum = parent
        if child     < last and items[maximum] < items[child]    : maximum = child
        if child + 1 < last and items[maximum] < items[child + 1]: maximum = child + 1
        if maximum == parent: break
        else: swap(items, parent, maximum); parent = maximum

def heapify(items, first, last):
    """Turn a list of items into a binary max heap. From Williams, 1964.
    Modeled after Sklearn's implementation at:
        https://github.com/scikit-learn/scikit-learn/blob/95d4f0841d57e8b5f6b2a570312e9d832e69debc/sklearn/tree/_splitter.pyx#L571
    
    Arguments:
        items      (list): The numbers to be sorted.
        first, last (int): The range of items to be sorted. 
    """
    parent = first + (last - first - 2)//2
    while True:
        sift_down(items, first, last, parent)
        if parent == first: break
        parent -= 1

def heapsort(items, first, last):
    """Applies the heapsort algorithm to sort a list of items from least to greatest. 
    From Williams, 1964. Modeled after Sklearn's implementation at:
        https://github.com/scikit-learn/scikit-learn/blob/95d4f0841d57e8b5f6b2a570312e9d832e69debc/sklearn/tree/_splitter.pyx#L571
    
    Arguments:
        items      (list): The numbers to be sorted.
        first, last (int): The range of items to be sorted. 
    """
    heapify(items, first, last); last-= 1
    while last > first: 
        swap(items, first, last)
        sift_down(items, first, last, first); last -= 1

In [22]:
rng = np.random.default_rng(42)

In [23]:
%timeit heapsort(rng.uniform(np.arange(1,225001), 225000), 0, 225000)

3.56 s ± 36.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### My Refactor of Sklearn
Below I experimented with a variation on Sklearn's approach, but it didn't speed things up.

In [24]:
def sift_down(items, first, last, parent):
    child = 2*parent + 1 # left child of the current parent node
    while child < last - first:
        if child + 1 < last - first and items[first + child] < items[first + child + 1]: # right child
            child += 1
        if items[first + child] > items[first + parent]:
            swap(items, first + child, first + parent)
        parent = child
        child = 2*parent + 1
            
def sort_heap(items, first, last):
    last -= 1
    while last - first > 0:
        swap(items, first, last)
        sift_down(items, first, last, first)
        last -= 1
          
def heapify(items, first, last):
    length = last - first
    last_parent = (length - 2)//2
    while True:
        sift_down(items, first, last, last_parent)
        if last_parent == first: break
        last_parent -= 1

def heapsort(items, first, last):
    heapify(items, first, last)
    sort_heap(items, first, last)

In [25]:
rng = np.random.default_rng(42)

In [26]:
%timeit heapsort(rng.uniform(np.arange(1,225001), 225000), 0, 225000)

4.27 s ± 32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Libstdc++'s [approach](https://github.com/gcc-mirror/gcc/blob/d9375e490072d1aae73a93949aa158fcd2a27018/libstdc%2B%2B-v3/include/bits/stl_algo.h#L1917)

In [27]:
def sift_down(items, first, hole_index, length, value):
    """Move an element in a heap downward so that its parent's value
    is greater than or equal to its own.
    
    The process is to swap a given item `hole_index` through its children all 
    the way to the bottom of the heap (towards higher indices). After that, swap it 
    back up through it's now-parent nodes towards the root (towards lower indices),
    so that at its final position, its parent has greater or equal value and 
    its children are smaller.
    
    Modeled after libstdc++ implementation at:
        https://github.com/gcc-mirror/gcc/blob/16e2427f50c208dfe07d07f18009969502c25dc8/libstdc%2B%2B-v3/include/bits/stl_heap.h#L219
        https://github.com/gcc-mirror/gcc/blob/16e2427f50c208dfe07d07f18009969502c25dc8/libstdc%2B%2B-v3/include/bits/stl_heap.h#L130
    
    Advantage of the libstdc++ implementation is that although it uses more 
    swaps, it requires less comparisons inside its while-loop. Runs faster
    than other approaches, such Sklearn's old design here:
        https://github.com/scikit-learn/scikit-learn/blob/95d4f0841d57e8b5f6b2a570312e9d832e69debc/sklearn/tree/_splitter.pyx#L548
    """
    # Take the parent at `hole_index` and swap it with
    # its children, generation by generation, until 
    # it is at a position among the items being looked
    # at (`length`) that has no children that are also
    # in that group of items.
    top_index, second_child = hole_index, hole_index
    while second_child < (length-1)//2:
        # Swap a parent with its largest child. Then do the 
        # same for the child of that largest child, so on and 
        # so forth, checking no further than the third item 
        # from end of the list.
        second_child = 2*(second_child + 1)
        if items[first + second_child] < items[first + second_child - 1]:
            second_child -= 1
        swap(items, first + hole_index, first + second_child)
        hole_index = second_child
    if length % 2 == 0 and second_child == (length - 2)//2:
        # Special case where number of items being looked at
        # is even and the second child currently being looked
        # at also happens to be the last parent in the list, 
        # with two children of its own. 
        #
        # In this scenario, swap the second child's parent 
        # with the second child's own first child.
        second_child = 2*(second_child + 1)
        swap(items, first + hole_index, first + second_child - 1)
        hole_index = second_child - 1
    # The parent that was initially located at `top_index` has
    # been swapped back through its descendants and now sits 
    # further back at the index currently stored in `hole_index`.
    #
    # Now, we'll swap this parent back up through its family tree, 
    # but *no further* than either its initial index (`top_index`) 
    # or to the index currently held by the furthest of a lineage 
    # of ancestors that all have values less than this element, 
    # whichever one comes first.
    parent = (hole_index - 1)//2
    while hole_index > top_index and items[first + parent] < value:
        swap(items, first + hole_index, first + parent)
        hole_index = parent
        parent = (hole_index - 1)//2

def heapify(items, first, last):
    """
    Modeled after libstdc++ implementation at:
        https://github.com/gcc-mirror/gcc/blob/16e2427f50c208dfe07d07f18009969502c25dc8/libstdc%2B%2B-v3/include/bits/stl_heap.h#L336
    """
    length = last - first
    parent = (length - 2)//2
    while True:
        parent_value = items[first + parent]
        sift_down(items, first, parent, length, parent_value)
        if parent == 0: return
        parent -= 1
            
def sort_heap(items, first, last):
    """
    Modeled after libstdc++ implementation at:
        https://github.com/gcc-mirror/gcc/blob/16e2427f50c208dfe07d07f18009969502c25dc8/libstdc%2B%2B-v3/include/bits/stl_heap.h#L436
        https://github.com/gcc-mirror/gcc/blob/16e2427f50c208dfe07d07f18009969502c25dc8/libstdc%2B%2B-v3/include/bits/stl_heap.h#L313
    """
    while last - first > 1:
        last -= 1
        last_value = items[last]
        swap(items, first, last)
        sift_down(items, first, first, last-first, last_value)

def heapsort(items, first, last):
    """
    Modeled after libstdc++ implementation at:
        https://github.com/gcc-mirror/gcc/blob/d9375e490072d1aae73a93949aa158fcd2a27018/libstdc%2B%2B-v3/include/bits/stl_algo.h#L1915
    """
    heapify(items, first, last)
    sort_heap(items, first, last)

In [28]:
rng = np.random.default_rng(42)

In [29]:
%timeit heapsort(rng.uniform(np.arange(1,225001), 225000), 0, 225000)

3.26 s ± 27.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%timeit heapsort(generate_med3_killer(225000), 0, 225000)

1.65 s ± 4.99 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Numpy's [approach](https://github.com/numpy/numpy/blob/084d05a5d1ef3efe79474b09b42594ee9ef086cb/numpy/core/src/npysort/heapsort.cpp#L51)

In [31]:
def sift_down(items, start, n, p, c, val):
    """Swap a heap item with one of its children if that child's value is 
    greater than or equal to that parent's value. From Williams, 1964.
    Modeled after Numpy's implementation at:
        https://github.com/numpy/numpy/blob/084d05a5d1ef3efe79474b09b42594ee9ef086cb/numpy/core/src/npysort/heapsort.cpp#L61
    
    Arguments:
        items        (list): 1-d array containing numbers.
        start         (int): Index of the first number.
        n             (int): Quantity of numbers.
        p             (int): Index of the parent.
        c             (int): Index of the parent's first (left) child.
        val: (float or int): The parent's value.
    """
    while c < n:    # Look at the descendents of current parent, `p`.
        if c < n-1 and items[start + c] < items[start + c + 1]: # Find larger of the first and second children.
            c += 1
        if val < items[start + c]: # If child greater than parent, swap child and parent.
            items[start + p] = items[start + c]
            p = c   # Current greater child becomes the parent.
            c += c  # Look at this child's child, if it exists.
        else:
            break 
    items[start + p] = val

def sort_heap(items, start, n):
    """Sort a binary max heap of numbers. From Williams, 1964.
    Modeled after Numpy's implementation at:
        https://github.com/numpy/numpy/blob/084d05a5d1ef3efe79474b09b42594ee9ef086cb/numpy/core/src/npysort/heapsort.cpp#L77
    
    Arguments:
        items (list): 1-d array containing the numbers to be sorted.
        start  (int): Index of the first number to be sorted.
        n      (int): Quantity of numbers to be sorted
    """
    while n > 0:
        n -= 1
        val = items[start + n]
        items[start + n] = items[start]
        sift_down(items, start, n, 0, 1, val)

def heapify(items, start, n):
    """Turn a list of items into a binary max heap. From Williams, 1964.
    Modeled after Numpy's implementation at:
        https://github.com/numpy/numpy/blob/084d05a5d1ef3efe79474b09b42594ee9ef086cb/numpy/core/src/npysort/heapsort.cpp#L59
    
    Arguments:
        items (list): 1-d array containing numbers.
        start  (int): Index of the first number.
        n      (int): Quantity of numbers.
    """
    for p in range((n-2)//2, -1, -1):
        val = items[start + p] # value of last parent
        sift_down(items, start, n, p, 2*p + 1, val)

def heapsort(items, start, n):
    """Applies the heapsort algorithm to sort a list of items from least to greatest. 
    From Williams, 1964.
    Arguments:
        items (list): 1-d array containing the numbers to be sorted.
        start  (int): Index of the first number to be sorted.
        n      (int): Quantity of numbers to be sorted
    """
    heapify(items, start, n)
    sort_heap(items, start, n)

In [32]:
rng = np.random.default_rng(42)

In [33]:
%timeit heapsort(rng.uniform(np.arange(1,225001), 225000), 0, 225000)

3.04 s ± 153 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
%timeit heapsort(generate_med3_killer(225000), 0, 225000)

1.57 s ± 115 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Numpy's design runs the fastest in Python and it's the most concise, so I'll go with that.

#### Turning `quicksort()` into `introsort()`
To transform quicksort into introsort, we need to make two modifications to the recursive loop:
1. Keep track of recursion depth by counting down backwards from the number we set as "maximum depth." When depth gets to zero, use heapsort to finish sorting the current partition.
2. Because enforcing a maximum depth is sufficient to prevent stack depth from exceeding $O(\log N)$, we no longer need to ensure that only the smaller sub-parition is sent to the next recursion call.

In [35]:
def introsort_loop(items, first, last, depth):
    """The recursive heart of the introsort algorithm.
    
    Arguments:
        items      (list): The numbers to be sorted.
        first, last (int): The range of items to be sorted. 
    """
    MIN_SIZE_THRESH = 16
    while last-first > MIN_SIZE_THRESH:
        if depth == 0:
            heapsort(items, first, last-first)
        depth -= 1
        med_three(items, first, last)
        cut = partition(items, first+1, last, first)
        introsort_loop(items, cut, last, depth)
        last = cut

The final adjustment is to decide what to choose as the maximum recursion depth. Musser said he preferred $2*\log_2 N$ because it worked well for his experiments. [Numpy](https://github.com/numpy/numpy/blob/5ffb84c3057a187b01acdeaa628137193df12098/numpy/core/src/npysort/quicksort.cpp#L160), [libstdc++](https://github.com/gcc-mirror/gcc/blob/d9375e490072d1aae73a93949aa158fcd2a27018/libstdc%2B%2B-v3/include/bits/stl_algo.h#L1958), and [Sklearn](https://github.com/scikit-learn/scikit-learn/blob/95d4f0841d57e8b5f6b2a570312e9d832e69debc/sklearn/tree/_utils.pyx#L77) (in its old introsort implementation) all do this as well.

In [36]:
def introsort(items, first, last):
    """Implementation as described in Musser, 1997. Switches to heapsort
    when max recursion depth exceeded. Otherwise uses median-of-three 
    quicksort (Bentley & McIlroy, 1993) with all the usual optimizations:
        - Swap equal elements.
        - Only process partitions longer than the minimum size threshold.
        - When a new partition is made, recurse on the smaller half and 
          iterate over the larger half.
        - Make a final pass with insertion sort over the entire list.

    Arguments:
        items      (list): The numbers to be sorted.
        first, last (int): The range of items to be sorted. 
    """
    introsort_loop(items, first, last, int(2*np.log2(last-first)))
    insertion_sort(items, first, last)

In [37]:
rng = np.random.default_rng(42)

In [38]:
introsort(rng.uniform(np.arange(1,225001), 225000), 0, 225000)

#### Introsort Never Goes Quadratic
Recall that my Python quicksort function took about 150 times longer to sort a median-of-three killer sequence.

In [39]:
rng = np.random.default_rng(42)

In [40]:
%timeit introsort(rng.uniform(np.arange(1,225001), 225000), 0, 225000)

1.69 s ± 84.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
%timeit introsort(np.array(generate_med3_killer(225000), dtype='float64'), 0, 225000)

5.42 s ± 55.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


However, the slowdown is much more reasonable with introsort, seeing as it's merely three times slower when faced with a median-of-three killer. 

## Implementing Introsort in Cython
On a lark, I translated my pure Python introsort routine into Cython to see how it's performance would fare against Sklearn's now-deprecated introsort Cython code.

In [42]:
%%cython
# cython: wraparound=False, boundscheck=False, cdivision=True, initializedcheck=False

import numpy as np
cimport numpy as np
ctypedef np.float64_t DTYPE_t
ctypedef np.intp_t SIZE_t

from libc.math cimport log as ln
cimport cython

cdef inline void swap(DTYPE_t* items, SIZE_t i, SIZE_t j) nogil:
    items[i], items[j] = items[j], items[i]

# Quicksort helpers

cdef inline void med_three(DTYPE_t* items, SIZE_t first, SIZE_t last) nogil:
    """Find the median-of-three pivot point of the second through final 
    items of a list of numbers. Once identified, the pivot is moved to 
    the front of the list. Borrows from libstdc++ implementation at: 
        https://github.com/gcc-mirror/gcc/blob/d9375e490072d1aae73a93949aa158fcd2a27018/libstdc%2B%2B-v3/include/bits/stl_algo.h#L78
    
    Arguments:
        items      : The numbers to be sorted.
        first, last: The range of items to be sorted. 
    """
    cdef SIZE_t middle = <int>(first + (last - first)/2)
    cdef SIZE_t second = first + 1
    last -= 1
    if items[second] < items[middle]:
        if items[middle] < items[last]:
            swap(items, first, middle)    
        elif items[second] < items[last]:
            swap(items, first, last)         
        else:                        
            swap(items, first, second)
    elif items[second] < items[last]:
        swap(items, first, second)
    elif items[middle] < items[last]:
        swap(items, first, last)
    else:
        swap(items, first, middle)

cdef inline SIZE_t partition(DTYPE_t* items, SIZE_t first, SIZE_t last, SIZE_t pivot) nogil:
    """Group numbers less than the pivot value together on the left and
    those that are greater on the right. Find the index that separates
    these two groups, which will belong to the first item that is greater
    than or equal to the pivot. Borrows from libstdc++ implementation at: 
        https://github.com/gcc-mirror/gcc/blob/d9375e490072d1aae73a93949aa158fcd2a27018/libstdc%2B%2B-v3/include/bits/stl_algo.h#L1885
    
    Arguments:
        items      : The numbers to be sorted.
        first, last: The range of items to be sorted. 
        pivot      : Index holding the median pivot value.
        
    Returns:
        Index of cut point used to partition the items into two smaller sequences.
    """
    while True:
        while first < last and items[first] < items[pivot]:
            first += 1                      # Get index of first item greater than or equal to median-of-three pivot. 
        last -= 1
        while items[pivot] < items[last]:
            last -= 1                       # Get index of last item less than or equal to the pivot.
        if not (first < last): 
            return first                    # After swaps are done, return index of first item in right partition.
        
        swap(items, first, last)            # Swap the first item greater than or equal to the pivot with the
                                            # last item less than or equal to the pivot. 
        first += 1

cdef inline void insertion_sort(DTYPE_t* items, SIZE_t first, SIZE_t last) nogil:
    """Follows the spirit of the Numpy implementation at: 
        https://github.com/numpy/numpy/blob/5ffb84c3057a187b01acdeaa628137193df12098/numpy/core/src/npysort/quicksort.cpp#L211
    
    Arguments:
        items      : The numbers to be sorted.
        first, last: The range of items to be sorted. 
    """
    cdef SIZE_t i
    cdef SIZE_t j
    cdef SIZE_t k
    cdef DTYPE_t val
    for i in range(first+1, last):
        j = i
        k = i - 1
        val = items[i]
        while (j > first) and val < items[k]:
            items[j] = items[k]
            j-=1
            k-=1
        items[j] = val

# Heapsort

cdef inline void sift_down(DTYPE_t* items, SIZE_t start, int n, SIZE_t p, SIZE_t c, DTYPE_t val) nogil:
    """Swap a heap item with one of its children if that child's value is 
    greater than or equal to that parent's value. From Williams, 1964.
    Modeled after Numpy's implementation at:
        https://github.com/numpy/numpy/blob/084d05a5d1ef3efe79474b09b42594ee9ef086cb/numpy/core/src/npysort/heapsort.cpp#L61
    
    Arguments:
        items: 1-d array containing numbers.
        start: Index of the first number.
        n    : Quantity of numbers.
        p    : Index of the parent.
        c    : Index of the parent's first (left) child.
        val  : The parent's value.
    """
    while c < n:    # Look at the descendents of current parent, `p`.
        if c < n-1 and items[start + c] < items[start + c + 1]: # Find larger of the first and second children.
            c += 1
        if val < items[start + c]: # If child greater than parent, swap child and parent.
            items[start + p] = items[start + c]
            p = c   # Current greater child becomes the parent.
            c += c  # Look at this child's child, if it exists.
        else:
            break 
    items[start + p] = val

cdef inline void sort_heap(DTYPE_t* items, SIZE_t start, int n) nogil:
    """Sort a binary max heap of numbers. From Williams, 1964.
    Modeled after Numpy's implementation at:
        https://github.com/numpy/numpy/blob/084d05a5d1ef3efe79474b09b42594ee9ef086cb/numpy/core/src/npysort/heapsort.cpp#L77
    
    Arguments:
        items: 1-d array containing the numbers to be sorted.
        start: Index of the first number to be sorted.
        n    : Quantity of numbers to be sorted
    """
    cdef DTYPE_t val
    while n > 0:
        n -= 1
        val = items[start + n]
        items[start + n] = items[start]
        sift_down(items, start, n, 0, 1, val)

cdef inline void heapify(DTYPE_t* items, SIZE_t start, int n) nogil:
    """Turn a list of items into a binary max heap. From Williams, 1964.
    Modeled after Numpy's implementation at:
        https://github.com/numpy/numpy/blob/084d05a5d1ef3efe79474b09b42594ee9ef086cb/numpy/core/src/npysort/heapsort.cpp#L59
    
    Arguments:
        items: 1-d array containing numbers.
        start: Index of the first number.
        n    : Quantity of numbers.
    """
    cdef DTYPE_t val
    cdef SIZE_t p
    cdef SIZE_t last_p = (n-2)//2
    for p in range(last_p, -1, -1):
        val = items[start + p] # value of last parent
        sift_down(items, start, n, p, 2*p + 1, val)

cdef inline void heapsort(DTYPE_t* items, SIZE_t start, int n) nogil:
    """Applies the heapsort algorithm to sort a list of items from least to greatest. 
    From Williams, 1964.
    Arguments:
        items: 1-d array containing the numbers to be sorted.
        start: Index of the first number to be sorted.
        n    : Quantity of numbers to be sorted
    """
    heapify(items, start, n)
    sort_heap(items, start, n)
    
# Introsort 

cdef void introsort_loop(DTYPE_t* items, SIZE_t first, SIZE_t last, int depth) nogil:
    """The recursive heart of the introsort algorithm.
    
    Arguments:
        items      : The numbers to be sorted.
        first, last: The range of items to be sorted. 
        depth      : Current recursion depth.
    """
    cdef int MIN_SIZE_THRESH = 16
    cdef SIZE_t cut
    while last-first > MIN_SIZE_THRESH:
        if depth == 0:
            heapsort(items, first, last-first)
        depth -= 1
        med_three(items, first, last)
        cut = partition(items, first+1, last, first)
        introsort_loop(items, cut, last, depth)
        last = cut

# Log base-2 helper function. From Sklearn's implementation at:
#     https://github.com/scikit-learn/scikit-learn/blob/95d4f0841d57e8b5f6b2a570312e9d832e69debc/sklearn/tree/_utils.pyx#L7
cdef inline DTYPE_t log2(DTYPE_t x) nogil:
    return ln(x) / ln(2.0)

cdef void introsort(DTYPE_t* items, SIZE_t first, SIZE_t last) nogil:
    """Implementation as described in Musser, 1997. Switches to heapsort
    when max recursion depth exceeded. Otherwise uses median-of-three 
    quicksort (Bentley & McIlroy, 1993) with all the usual optimizations:
        - Swap equal elements.
        - Only process partitions longer than the minimum size threshold.
        - When a new partition is made, recurse on the smaller half and 
          iterate over the larger half.
        - Make a final pass with insertion sort over the entire list.

    Arguments:
        items      : The numbers to be sorted.
        first, last: The range of items to be sorted. 
    """
    cdef int max_depth = 2 * <int>log2(last-first)
    introsort_loop(items, first, last, max_depth)
    insertion_sort(items, first, last)
    
# Python wrapper
def sort(np.ndarray[np.float64_t, ndim=1, mode="c"] items):
    """Wrapper function for my Cython implementation of `introsort()`. 
    Followed Laura Mendoza's guide on how to have Cython access
    a Numpy array using the C pointer:
        https://members.loria.fr/LMendoza/link/Cython_speedup_notes.html#Working-with-numpy-arrays-in-I/O
    
    Arguments:
        items (Numpy array, float64): The numbers to be sorted.
    """
    cdef np.float64_t* items_ptr
    items_ptr = <np.float64_t*> items.data
    cdef SIZE_t first = 0
    cdef SIZE_t last = items.size
    introsort(items_ptr, first, last)

#### My Cython Introsort's Speed

In [43]:
rng = np.random.default_rng(42)

In [44]:
%timeit sort(rng.uniform(np.arange(1,225001), 225000))

15.6 ms ± 295 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [45]:
%timeit sort(np.array(generate_med3_killer(225000), dtype='float64'))

48.1 ms ± 1.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Numpy's Speed

In [46]:
rng = np.random.default_rng(42)

In [47]:
%timeit np.sort(rng.uniform(np.arange(1,225001), 225000))

16.4 ms ± 284 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [48]:
%timeit np.sort(np.array(generate_med3_killer(225000), dtype='float64'))

43.9 ms ± 996 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Aside: Passing Numpy Arrays to Cython Functions
By far the trickiest part of converting my pure Python introsort implementation to Cython was figuring out how to properly pass a Numpy array to my new Cython functions. In the end, I followed the approach recommended by [Laura Mendoza](https://members.loria.fr/LMendoza/), which is to define and pass a C pointer to a Numpy array. Laura ran a set of [speed tests](https://members.loria.fr/LMendoza/link/Cython_speedup_notes.html#Working-with-numpy-arrays-in-I/O) comparing the various ways of handing Numpy arrays off to Cython and found that C pointers were fastest.

Another viable approach would be to use a memoryview. [DavidW](https://stackoverflow.com/users/4657412/davidw) explained how to do this in a StackOverflow [answer](https://stackoverflow.com/questions/10718699/convert-numpy-array-to-cython-pointer/54832269#54832269), and here's how my `introsort()` wrapper would look if I had used memoryviews instead of C pointers:

```python
def sort(items):
    """Wrapper function for my Cython `introsort()` function. DavidW's
    StackOverflow shows how to define the memoryview of the `items` 
    Numpy array: https://stackoverflow.com/a/54832269/8628758
    
    Arguments:
        items (Numpy array, float64): The numbers to be sorted.
    """
    cdef np.float64_t[::1] items_memoryview_buffer = np.ascontiguousarray(values, dtype = np.float64)
    cdef np.float64_t* items_buffer = &items_memoryview_buffer[0]
    cdef SIZE_t first = 0
    cdef SIZE_t last = items.size
    introsort(items_buffer, first, last)
```

## In Conclusion: Introsort is the Best
The speed of my home-grown Cython version of introsort is nearly identical to Numpy's `np.sort()`. This shouldn't be surprising, seeing as how both my and Numpy's introsort implementations use heapsort to pretect against deep recursion and use insertion sort to handle small partitions.

On the other hand, the biggest bombshell finding of this notebook was that Sklearn's [newly refactored](https://github.com/scikit-learn/scikit-learn/pull/22868) sorting code blows up and goes quadratic when facing a median-of-three killer sequence. This is because it relies recursive calls to quicksort regardless of how frequently the median-of-three algorithm creates lopsided partitions for a given sequence of numbers. Too many lopsided partitions leads necessitates deeper recursion, which makes compute time go quadratic in the worst-case scenario.

Sklearn's older sorting implementation fares much better on killer sequences, as it used heapsort to prevent catastrophic slowdowns. However, even when sorting randomly drawn sequences of numbers, both Sklearn sort routines are nearly twice as slow as my Cython code. I hypothesize that this is partly due to the fact that my code doesn't use quicksort for partitions containing less than 16 numbers, while Sklearn's newer algorithm uses quicksort for all partitions containing four or more numbers. What's more, Sklearn's older sort function uses quicksort for *all* partition sizes, no matter how small. 

#### Final Aside:  How much does using insertion sort really help?
I believe that at least part of the reason my sort function is faster than either of Sklearn's is because it doesn't use quicksort on partitions holding less than 16 numbers, instead leaving them all to be mopped up by a final pass of insertion sort. However, it's also reasonable to assume that part of Sklearn's slowdown stems from the fact that it had to perform twice the number of swaps, given that it's designed to concurrently argsort a list of indices as it sorts the list of numbers.

So, on a final lark, I modified my Cython introsort implementation to dual sort row indices along with raw numerical values just like Sklearn does in order to test it out and see if it still runs faster than Sklearn's sort functions.

In [49]:
%%cython
# cython: wraparound=False, boundscheck=False, cdivision=True, initializedcheck=False

import numpy as np
cimport numpy as np
ctypedef np.float64_t DTYPE_t
ctypedef np.intp_t SIZE_t

from libc.math cimport log as ln
cimport cython

cdef inline void dual_swap(DTYPE_t* items, SIZE_t* rows, SIZE_t i, SIZE_t j) nogil:
    items[i], items[j] = items[j], items[i]
    rows[i], rows[j] = rows[j], rows[i]

# Quicksort helpers

cdef inline void dual_med_three(DTYPE_t* items, SIZE_t* rows, SIZE_t first, SIZE_t last) nogil:
    """Find the median-of-three pivot point of the second through final 
    items of a list of numbers. Once identified, the pivot is moved to 
    the front of the list. Borrows from libstdc++ implementation at: 
        https://github.com/gcc-mirror/gcc/blob/d9375e490072d1aae73a93949aa158fcd2a27018/libstdc%2B%2B-v3/include/bits/stl_algo.h#L78
    
    Arguments:
        items      : The numbers to be sorted.
        rows       : Row indices of all training samples.
        first, last: The range of items to be sorted. 
    """
    cdef SIZE_t middle = <int>(first + (last - first)/2)
    cdef SIZE_t second = first + 1
    last -= 1
    if items[second] < items[middle]:
        if items[middle] < items[last]:
            dual_swap(items, rows, first, middle)    
        elif items[second] < items[last]:
            dual_swap(items, rows, first, last)         
        else:                        
            dual_swap(items, rows, first, second)
    elif items[second] < items[last]:
        dual_swap(items, rows, first, second)
    elif items[middle] < items[last]:
        dual_swap(items, rows, first, last)
    else:
        dual_swap(items, rows, first, middle)

cdef inline SIZE_t dual_partition(DTYPE_t* items, SIZE_t* rows, SIZE_t first, SIZE_t last, SIZE_t pivot) nogil:
    """Group numbers less than the pivot value together on the left and
    those that are greater on the right. Find the index that separates
    these two groups, which will belong to the first item that is greater
    than or equal to the pivot. Borrows from libstdc++ implementation at: 
        https://github.com/gcc-mirror/gcc/blob/d9375e490072d1aae73a93949aa158fcd2a27018/libstdc%2B%2B-v3/include/bits/stl_algo.h#L1885
    
    Arguments:
        items      : The numbers to be sorted.
        rows       : Row indices of all training samples.
        first, last: The range of items to be sorted. 
        pivot      : Index holding the median pivot value.
        
    Returns:
        Index of cut point used to partition the items into two smaller sequences.
    """
    while True:
        while first < last and items[first] < items[pivot]:
            first += 1                      # Get index of first item greater than or equal to median-of-three pivot. 
        last -= 1
        while items[pivot] < items[last]:
            last -= 1                       # Get index of last item less than or equal to the pivot.
        if not (first < last): 
            return first                    # After swaps are done, return index of first item in right partition.
        
        dual_swap(items, rows, first, last)      # Swap the first item greater than or equal to the pivot with the
                                            # last item less than or equal to the pivot. 
        first += 1

cdef inline void dual_insertion_sort(DTYPE_t* items, SIZE_t* rows, SIZE_t first, SIZE_t last) nogil:
    """Follows the spirit of the Numpy implementation at: 
        https://github.com/numpy/numpy/blob/5ffb84c3057a187b01acdeaa628137193df12098/numpy/core/src/npysort/quicksort.cpp#L211
    
    Arguments:
        items      : The numbers to be sorted.
        rows       : Row indices of all training samples.
        first, last: The range of items to be sorted. 
    """
    cdef SIZE_t i
    cdef SIZE_t j
    cdef SIZE_t k
    cdef DTYPE_t val
    for i in range(first+1, last):
        j = i
        k = i - 1
        val = items[i]
        row = rows[i]
        while (j > first) and val < items[k]:
            items[j] = items[k]
            rows[j] = rows[k]
            j-=1
            k-=1
        items[j] = val
        rows[j] = row

# Heapsort

cdef inline void dual_sift_down(DTYPE_t* items, SIZE_t* rows, SIZE_t start, int n, 
                                SIZE_t p, SIZE_t c, DTYPE_t val, SIZE_t row) nogil:
    """Swap a heap item with one of its children if that child's value is 
    greater than or equal to that parent's value. From Williams, 1964.
    Modeled after Numpy's implementation at:
        https://github.com/numpy/numpy/blob/084d05a5d1ef3efe79474b09b42594ee9ef086cb/numpy/core/src/npysort/heapsort.cpp#L61
    
    Arguments:
        items: 1-d array containing numbers.
        rows : Row indices of all training samples.
        start: Index of the first number.
        n    : Quantity of numbers.
        p    : Index of the parent.
        c    : Index of the parent's first (left) child.
        val  : The parent's value.
        row  : The parent's training row index.
    """
    while c < n:    # Look at the descendents of current parent, `p`.
        if c < n-1 and items[start + c] < items[start + c + 1]: # Find larger of the first and second children.
            c += 1
        if val < items[start + c]: # If child greater than parent, swap child and parent.
            items[start + p] = items[start + c]
            rows[start + p] = rows[start + c]
            p = c   # Current greater child becomes the parent.
            c += c  # Look at this child's child, if it exists.
        else:
            break 
    items[start + p] = val
    rows[start + p] = row

cdef inline void dual_sort_heap(DTYPE_t* items, SIZE_t* rows, SIZE_t start, int n) nogil:
    """Sort a binary max heap of numbers. From Williams, 1964.
    Modeled after Numpy's implementation at:
        https://github.com/numpy/numpy/blob/084d05a5d1ef3efe79474b09b42594ee9ef086cb/numpy/core/src/npysort/heapsort.cpp#L77
    
    Arguments:
        items: 1-d array containing the numbers to be sorted.
        rows : Row indices of all training samples.
        start: Index of the first number to be sorted.
        n    : Quantity of numbers to be sorted
    """
    cdef DTYPE_t val
    cdef SIZE_t row
    while n > 0:
        n -= 1
        val = items[start + n]
        row = rows[start + n]
        items[start + n] = items[start]
        rows[start + n] = rows[start]
        dual_sift_down(items, rows, start, n, 0, 1, val, row)

cdef inline void dual_heapify(DTYPE_t* items, SIZE_t* rows, SIZE_t start, int n) nogil:
    """Turn a list of items into a binary max heap. From Williams, 1964.
    Modeled after Numpy's implementation at:
        https://github.com/numpy/numpy/blob/084d05a5d1ef3efe79474b09b42594ee9ef086cb/numpy/core/src/npysort/heapsort.cpp#L59
    
    Arguments:
        items: 1-d array containing numbers.
        rows : Row indices of all training samples.
        start: Index of the first number.
        n    : Quantity of numbers.
    """
    cdef DTYPE_t val
    cdef SIZE_t p
    cdef SIZE_t last_p = (n-2)//2
    for p in range(last_p, -1, -1):
        val = items[start + p] # value of last parent
        row = rows[start + p]
        dual_sift_down(items, rows, start, n, p, 2*p + 1, val, row)

cdef inline void dual_heapsort(DTYPE_t* items, SIZE_t* rows, SIZE_t start, int n) nogil:
    """Applies the heapsort algorithm to sort a list of items from least to greatest. 
    From Williams, 1964.
    Arguments:
        items: 1-d array containing the numbers to be sorted.
        rows : Row indices of all training samples.
        start: Index of the first number to be sorted.
        n    : Quantity of numbers to be sorted
    """
    dual_heapify(items, rows, start, n)
    dual_sort_heap(items, rows, start, n)
    
# Introsort 

cdef void dual_introsort_loop(DTYPE_t* items, SIZE_t* rows, SIZE_t first, SIZE_t last, int depth) nogil:
    """The recursive heart of the introsort algorithm.
    
    Arguments:
        items      : The numbers to be sorted.
        rows       : Row indices of all training samples.
        first, last: The range of items to be sorted. 
        depth      : Current recursion depth.
    """
    cdef int MIN_SIZE_THRESH = 16
    cdef SIZE_t cut
    while last-first > MIN_SIZE_THRESH:
        if depth == 0:
            dual_heapsort(items, rows, first, last-first)
        depth -= 1
        dual_med_three(items, rows, first, last)
        cut = dual_partition(items, rows, first+1, last, first)
        dual_introsort_loop(items, rows, cut, last, depth)
        last = cut

# Log base-2 helper function. From Sklearn's implementation at:
#     https://github.com/scikit-learn/scikit-learn/blob/95d4f0841d57e8b5f6b2a570312e9d832e69debc/sklearn/tree/_utils.pyx#L7
cdef inline DTYPE_t log2(DTYPE_t x) nogil:
    return ln(x) / ln(2.0)

cdef void dual_introsort(DTYPE_t* items, SIZE_t* rows, SIZE_t first, SIZE_t last) nogil:
    """Implementation as described in Musser, 1997. Switches to heapsort
    when max recursion depth exceeded. Otherwise uses median-of-three 
    quicksort (Bentley & McIlroy, 1993) with all the usual optimizations:
        - Swap equal elements.
        - Only process partitions longer than the minimum size threshold.
        - When a new partition is made, recurse on the smaller half and 
          iterate over the larger half.
        - Make a final pass with insertion sort over the entire list.

    Arguments:
        items      : The numbers to be sorted.
        rows       : Row indices of all training samples.
        first, last: The range of items to be sorted. 
    """
    cdef int max_depth = 2 * <int>log2(last-first)
    dual_introsort_loop(items, rows, first, last, max_depth)
    dual_insertion_sort(items, rows, first, last)
    
# Python wrapper
def dual_sort(np.ndarray[np.float64_t, ndim=1, mode="c"] items, np.ndarray[np.intp_t, ndim=1, mode="c"] rows,
              node_start_idx):
    """Wrapper function for my Cython implementation of `introsort()`. 
    Followed Laura Mendoza's guide on how to have Cython access
    a Numpy array using the C pointer:
        https://members.loria.fr/LMendoza/link/Cython_speedup_notes.html#Working-with-numpy-arrays-in-I/O
    
    Arguments:
        items (Numpy array, float64): The numbers to be sorted.
        rows      (Numpy array, int): Training set row indices of all samples used to 
                                      fit the decision tree.
        node_start_idx         (int): Index of the list of training rows that marks
                                      the beginning of the current decision tree node.
    """
    cdef np.float64_t* items_ptr
    items_ptr = <np.float64_t*> items.data
    cdef np.intp_t* rows_ptr
    rows_ptr = <np.intp_t*> rows.data
    cdef SIZE_t first = 0
    cdef SIZE_t last = items.size
    cdef SIZE_t start = node_start_idx
    dual_introsort(items_ptr, rows_ptr + start, first, last)

Recall that Sklearn's new sorting logic sorted a list of 225K randomly chosen numbers in `34.7 ms` while their older implementation accomplished this in `39.1 ms`. 

Now let's see how my Cython dual introsort performs:

In [50]:
rng = np.random.default_rng(42)

In [51]:
%timeit dual_sort(rng.uniform(np.arange(1,225001), 225000), np.array([i for i in range(225000)]), 0)

35.9 ms ± 393 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


It's clear that having to do those dual swaps of raw feature values and row indices slows my code down considerably. Nonetheless, it appears that using insertion sort helps my Cython code to still be neck and neck with Sklearn's ["new and improved"](https://twitter.com/thomasjpfan/status/1506043093107318788) sorting logic. And unlike Sklearn, there's no risk of my sort function ever going quadratic and slowing to a crawl!

## Epilogue
Not more than two months after Sklearn's new sorting logic was merged, a bug report [was filed](https://github.com/scikit-learn/scikit-learn/issues/23397) that described an 8x slowdown when using the latest version of the library to fit a decision tree classifier to data that contained integer-encoded low-cardinality categorical features.

Sklearn integer-encodes unordered variables by replacing levels with consecutive integers, and then treating the encoded columns as run-of-the-mill numerical features during tree fitting. So for example, if you had a feature that contained months:

```
['June', 'August', 'December', 'May', 'January', ...]
```

Sklearn would map these levels to integers, and before training began, the column would look something like:

```
[5, 7, 11, 4, 0, ...]
```

If such a column had hundreds or thousands of rows, it's not difficult to imagine that several subsequences may have been not too dissimilar to a median-of-3 killer sequence:

In [53]:
generate_med3_killer(12)

[1, 7, 3, 9, 5, 11, 2, 4, 6, 8, 10, 12]

And though the 8x slowdown that the bug reported was nowhere near the 200x nightmare slowdown that we previously saw Sklearn undergo when it was faced with true-blue median-of-3 killer sequence of epic proportions, I can't help but wonder if loosely similar dynamics may have been at play.

Sure enough, [reverting to the old sorting logic](https://github.com/scikit-learn/scikit-learn/pull/23410) resolved the problem, and this example would seem to lend anecdotal support to Musser's presumption that median-of-3 killer sequences (or close relatives thereof) occur more commonly in the real world than might otherwise be expected if one assumed their presence (or absence) was governed by a uniform distribution.